In [ ]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def open_json(filename):
    with open(filename, "r", encoding="utf-8") as read_file:
        data = json.load(read_file)
    return data

In [3]:
class Person:
    def __init__(self, dictionary):
        self.id = dictionary['id']
        self.name = dictionary['name']
        self.gender = dictionary['gender']
        if 'pids' in dictionary.keys():
            self.pids = dictionary['pids']
        if 'mid' in dictionary.keys():
            self.mid = dictionary['mid']
        if 'fid' in dictionary.keys():
            self.fid = dictionary['fid']
        self.cids = []
        self.rank = -1

    def __repr__(self):
        result = str(self.id) + ', ' + self.name + ', ' + self.gender
        if hasattr(self, 'pids'):
            result += ', ' + str(self.pids)
        if hasattr(self, 'mid'):
            result += ', ' + str(self.mid)
        if hasattr(self, 'fid'):
            result += ', ' + str(self.fid)
        if len(self.cids) != 0:
            result += ', ' + str(self.cids)
        result += ', ' + str(self.rank)
        return '{' + result + '}'

In [4]:
def get_all_rank(tree, rank):
    result = []
    for id in tree:
        if tree[id].rank == str(rank):
            result.append(tree[id])
    return result

In [77]:
class FamilyTree:
    def __init__(self, tree_list):
        self.tree = {}
        if type(tree_list) is list:
            for node in tree_list:
                if type(node) is dict:
                    person = Person(node)
                    self.tree.update({person.id: person})
                else:
                    print("List must contains dict!")
        else:
            print("Tree must be list!")
        self.__add_children()
        self.__add_rank()

    def __add_children(self):
        result = self.tree
        for i in self.tree:
            if hasattr(self.tree[i], "mid"):
                result[self.tree[i].mid].cids.append(self.tree[i].id)
            if hasattr(self.tree[i], "fid"):
                result[self.tree[i].fid].cids.append(self.tree[i].id)
        self.tree = result

    def __add_rank(self):
        result = self.tree
        def add_rank_node(node, rank):
            if node.rank == -1:
                node.rank = str(rank)
                if hasattr(node, 'pids'):
                    for n in node.pids:
                        add_rank_node(result[n], rank)
            else:
                rank+=1
                if len(node.cids) != 0:
                    for n in node.cids:
                        add_rank_node(result[n], rank)
        add_rank_node(result[1], 0)
        self.tree = result

    def get_all_rank(self):
        ranks = []
        for person in self.tree.values():
            ranks.append(int(person.rank))
        return ranks

    def get_all_name(self):
        names = []
        for person in self.tree.values():
            names.append(person.name.split(" ")[0])
        return names

In [16]:
family_tree = FamilyTree(open_json('data/tree3_very_simple.json'))
family_tree.tree

{1: {1, Михаил Федорович, male, [2], [3], 0, -1},
 2: {2, Евдокия Лукьяновна Стрешнева, female, [1], [3], 0, -1},
 3: {3, Алексей Михайлович, male, 1, 2, 1, -1}}